# Legacy AISTATS Simulations

Original AISTATS experiment notebook, updated to use the `models/` package.
Compares UCB, Vanilla SAE, Horizontal/Vertical LSAE, and Scheduled SP2.

In [ ]:
import sys, os
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from runners import run_episodes
from plotting.utils import plot_and_save
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import random

%load_ext autoreload
%autoreload 2

In [ ]:
config = {
    'k' : 10,
    'm' : 20,
    'iters' : int(5e4),
    'episodes' : 100,
    'var' : 1,
    'c' : math.sqrt(1)
}

config['mu'] = 'random'
config['epsilons'] = np.array( [0.9] * int(config['m'] / 4) 
                              + [0.93] * int(config['m'] / 4) 
                              + [0.95] * int(config['m'] / 4) 
                              + [0.99] * int(config['m'] / 4) ) 

unique_epsilons = sorted(set(config['epsilons']))
epsilons_str = '-'.join(map(str, unique_epsilons))
epsilons_str = epsilons_str.replace('.', '_')

formatted_iters = f"{config['iters']:.0e}"
foldername = f"experiment_k{config['k']}_m{config['m']}_iters{formatted_iters}_epis{config['episodes']}_var{config['var']}_eps{epsilons_str}"

base_actions = np.random.randint(config['k'], size=(config['m'],))

print(foldername)
print(base_actions)

In [ ]:
vars = {}
algorithms = [
    ('SAE', True, 'Weighted'),
    ('SAE', True, 'Scheduled'),
    ('UCB', False, ''),
    ('SAE', False, ''),
    ('SAE', True, 'Horizontal'),
    ('SAE', True, 'Vertical')
]

In [ ]:
config['mu'] = [0.8] + [0] * int(config['k'] / 2 - 1) + [1.0] + [0] * (config['k'] - int(config['k'] / 2 - 1) - 2)
random.shuffle(config['mu'])
print(config['mu'])

In [ ]:
# Run with means defined above
for alg_name, rep, mode in algorithms:
    name, file_loc = run_episodes(alg_name, 
                rep, 
                mode, 
                iters=config['iters'], 
                k=config['k'], 
                episodes=config['episodes'], 
                m=config['m'], 
                var=config['var'],
                mu=config['mu'], 
                eps=config['epsilons'],
                folder_name=foldername,
                base_actions=base_actions)

    vars[name] = pickle.load(open(file_loc, "rb"))

In [ ]:
results = []
labels = []

for elt in vars:
    results.append(np.sum(vars[elt]['regret'], axis=1))
    labels.append(elt)
info = ' (K:'+str(config['k'])+', M: '+str(config['m']) + ', var: ' + str(config['var']) + ')'
plot_and_save(results, labels, info=info, f_name='results_'+foldername)